In [ ]:
!git clone https://github.com/icmc-data/BadGlobalMinima.git

fatal: destination path 'BadGlobalMinima' already exists and is not an empty directory.


In [ ]:
!pip install git+https://github.com/deepmind/dm-haiku


  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-e88v7bnc
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-e88v7bnc
  Created wheel for dm-haiku: filename=dm_haiku-0.0.5.dev0-cp37-none-any.whl size=530361 sha256=b817860f712a97607ae44c3ee85d095125ea5ade447ceb2a8a79147cb98078c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-dto5n17h/wheels/97/0f/e9/17f34e377f8d4060fa88a7e82bee5d8afbf7972384768a5499
Successfully built dm-haiku
  Found existing installation: dm-haiku 0.0.4
    Uninstalling dm-haiku-0.0.4:
      Successfully uninstalled dm-haiku-0.0.4


In [ ]:
import torch
import numpy as np
import jax.numpy as jnp
import jax
import haiku as hk
import copy

In [ ]:
def lp_path_norm_pt(model, device, p=2, input_size=[3, 32, 32]):
    tmp_model = copy.deepcopy(model)
    tmp_model.eval()

    with torch.no_grad():

        for param in tmp_model.parameters():
            if param.requires_grad:
                param.abs_().pow_(p)
        data_ones = torch.ones(input_size).to(device)
        # print(tmp_model.parameters)
    return (tmp_model(data_ones).sum() ** (1 / p )).item()

In [ ]:
 def lp_path_norm_jax(forward, weights, p=2, input_size=[3, 32, 32]):
    pw_model = jax.tree_map(lambda w: jnp.power(jnp.abs(w), p), weights)
    data_ones = jnp.ones(input_size)
    return (forward.apply(pw_model, state, data_ones).sum() ** (1 / p ))

In [ ]:
X = np.random.rand(10)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(10, 10)
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(10, 1)
        
    def forward(self, X):
        return self.l2(self.relu(self.l1(X)))

In [ ]:
model = Model()
model.forward(torch.Tensor(X))

tensor([-0.4074], grad_fn=<AddBackward0>)

In [ ]:
def forward(X):
    mlp = hk.Sequential([hk.Linear(10),  jax.nn.relu, hk.Linear(1)])
    return mlp(X)

In [ ]:
net = hk.without_apply_rng(hk.transform(forward))

In [ ]:
rng = jax.random.PRNGKey(42)

params = net.init(rng, X)

/usr/local/lib/python3.7/dist-packages/jax/_src/numpy/lax_numpy.py:3035: UserWarning: Explicitly requested dtype float64 requested in zeros is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  lax._check_user_dtype_supported(dtype, "zeros")


In [ ]:
params_dict['linear']['w'] = jnp.array(model.l1.weight.detach().numpy()).T

params_dict['linear']['b'] = jnp.array(model.l1.bias.detach().numpy()).T

params_dict['linear_1']['w'] = jnp.array(model.l2.weight.detach().numpy()).T

params_dict['linear_1']['b'] = jnp.array(model.l2.bias.detach().numpy()).T

In [ ]:
hk_pytorch_params = hk.data_structures.to_haiku_dict(params_dict)

In [ ]:
net.apply(hk_pytorch_params, jnp.array(X))

DeviceArray([-0.40736407], dtype=float32)

In [ ]:
lp_path_norm_pt(model, 'cpu', input_size=[10])

0.4097086191177368

In [ ]:
lp_path_norm_jax(net, hk_pytorch_params, input_size=[10])

DeviceArray(0.40970862, dtype=float32)